In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Fastai Cat-Dog Classifier

import os
from fastai.vision.all import *

# Set up path for data
path = Path("pet_images")

# Download and extract the dataset
if not path.exists():
    urls = ['https://github.com/fastai/fastbook/raw/master/images/cats.tgz',
            'https://github.com/fastai/fastbook/raw/master/images/dogs.tgz']
    for url in urls:
        download_url(url, path)
        tarfile.open(f'{path}/{url.split("/")[-1]}').extractall(path=path)

# Create a DataBlock
pets = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224)
)

# Create DataLoaders
dls = pets.dataloaders(path)

# Create and train the model
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

# Test the model
img = PILImage.create('path_to_test_image.jpg')
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[0]:.4f}")

# Save the model
learn.export('pet_classifier.pkl')

print("Model training complete and saved!")